In [1]:
%cd
%cd pololu-rpi-slave-arduino-library/pi
%ls -l

/home/pirate
/home/pirate/pololu-rpi-slave-arduino-library/pi
insgesamt 48
-rw-r--r-- 1 pirate pirate 1781 Feb 10 11:12 a_star.py
-rw-r--r-- 1 pirate pirate 3351 Feb 10 11:14 a_star.pyc
-rwxr-xr-x 1 pirate pirate 1694 Feb 10 11:12 a_star_slave_heartbeat.sh*
-rwxr-xr-x 1 pirate pirate 1671 Feb 10 11:12 a_star_slave.sh*
-rw-r--r-- 1 pirate pirate  160 Feb 10 11:12 beep.py
-rwxr--r-- 1 pirate pirate  622 Feb 10 11:12 benchmark.py*
-rw-r--r-- 1 pirate pirate  226 Feb 10 11:12 blink.py
-rwxr-xr-x 1 pirate pirate  396 Feb 10 11:12 heartbeat.py*
drwxr-xr-x 2 pirate pirate 4096 Feb 10 11:14 __pycache__/
-rwxr-xr-x 1 pirate pirate 1864 Feb 10 11:12 server.py*
drwxr-xr-x 2 pirate pirate 4096 Feb 10 11:12 static/
drwxr-xr-x 2 pirate pirate 4096 Feb 10 11:12 templates/


In [12]:
# %load beep.py
# Copyright Pololu Corporation.  For more information, see https://www.pololu.com/
from a_star import AStar

a_star = AStar()

a_star.play_notes("o4l16ceg>c8")


In [17]:
# %load blink.py
# Copyright Pololu Corporation.  For more information, see https://www.pololu.com/
from a_star import AStar
import time

a_star = AStar()
a = 0
steps = 20
while a < steps:
  a_star.leds(0,0,0)
  time.sleep(0.5)
  a_star.leds(1,1,1)
  time.sleep(0.5)
  a = a + 1


In [19]:
# %load server.py
#!/usr/bin/env python3

# Copyright Pololu Corporation.  For more information, see https://www.pololu.com/
from flask import Flask
from flask import render_template
from flask import redirect
from subprocess import call
app = Flask(__name__)
app.debug = True

from a_star import AStar
a_star = AStar()

import json

led0_state = False
led1_state = False
led2_state = False

@app.route("/")
def hello():
    return render_template("index.html")

@app.route("/status.json")
def status():
    buttons = a_star.read_buttons()
    analog = a_star.read_analog()
    battery_millivolts = a_star.read_battery_millivolts()
    encoders = a_star.read_encoders()
    data = {
        "buttons": buttons,
        "battery_millivolts": battery_millivolts,
        "analog": analog,
        "encoders": encoders
    }
    return json.dumps(data)

@app.route("/motors/<left>,<right>")
def motors(left, right):
    a_star.motors(int(left), int(right))
    return ""

@app.route("/leds/<int:led0>,<int:led1>,<int:led2>")
def leds(led0, led1, led2):
    a_star.leds(led0, led1, led2)
    global led0_state
    global led1_state
    global led2_state
    led0_state = led0
    led1_state = led1
    led2_state = led2
    return ""

@app.route("/heartbeat/<int:state>")
def hearbeat(state):
    if state == 0:
      a_star.leds(led0_state, led1_state, led2_state)
    else:
        a_star.leds(not led0_state, not led1_state, not led2_state)
    return ""

@app.route("/play_notes/<notes>")
def play_notes(notes):
    a_star.play_notes(notes)
    return ""

@app.route("/halt")
def halt():
    call(["bash", "-c", "(sleep 2; sudo halt)&"])
    return redirect("/shutting-down")

@app.route("/shutting-down")
def shutting_down():
    return "Shutting down in 2 seconds! You can remove power when the green LED stops flashing."

if __name__ == "__main__":
    app.run(host = "0.0.0.0")


In [2]:
from a_star import AStar
a_star = AStar()

In [23]:
a_star.motors(50,50)

In [24]:
a_star.motors(0,0)

In [25]:
a_star.motors(-50,-50)

In [26]:
a_star.motors(0,0)

Werte von 30 bis 300 (ToDo messen ob nach 300 die Drezahl erhöht wird)

In [57]:
print(a_star.read_encoders())

(-8905, -14134)


In [36]:
# %load a_star.py
# Copyright Pololu Corporation.  For more information, see https://www.pololu.com/
import smbus
import struct
import time

class AStar:
  def __init__(self):
    self.bus = smbus.SMBus(1)

  def read_unpack(self, address, size, format):
    # Ideally we could do this:
    #    byte_list = self.bus.read_i2c_block_data(20, address, size)
    # But the AVR's TWI module can't handle a quick write->read transition,
    # since the STOP interrupt will occasionally happen after the START
    # condition, and the TWI module is disabled until the interrupt can
    # be processed.
    #
    # A delay of 0.0001 (100 us) after each write is enough to account
    # for the worst-case situation in our example code.

    self.bus.write_byte(20, address)
    time.sleep(0.0001)
    byte_list = [self.bus.read_byte(20) for _ in range(size)]
    return struct.unpack(format, bytes(byte_list))

  def write_pack(self, address, format, *data):
    data_array = list(struct.pack(format, *data))
    self.bus.write_i2c_block_data(20, address, data_array)
    time.sleep(0.0001)

  def leds(self, red, yellow, green):
    self.write_pack(0, 'BBB', red, yellow, green)

  def play_notes(self, notes):
    self.write_pack(24, 'B15s', 1, notes.encode("ascii"))

  def motors(self, left, right):
    self.write_pack(6, 'hh', left, right)

  def read_buttons(self):
    return self.read_unpack(3, 3, "???")

  def read_battery_millivolts(self):
    return self.read_unpack(10, 2, "H")

  def read_analog(self):
    return self.read_unpack(12, 12, "HHHHHH")

  def read_encoders(self):
    return self.read_unpack(39, 4, 'hh')

  def test_read8(self):
    self.read_unpack(0, 8, 'cccccccc')

  def test_write8(self):
    self.bus.write_i2c_block_data(20, 0, [0,0,0,0,0,0,0,0])
    time.sleep(0.0001)


In [61]:
!yes | /usr/sbin/i2cdetect 1

WARNING! This program can confuse your I2C bus, cause data loss and worse!
I will probe file /dev/i2c-1.
I will probe address range 0x03-0x77.
Continue? [Y/n]      0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:          -- -- -- -- -- -- -- -- -- -- -- -- -- 
10: -- -- -- -- 14 -- -- -- -- -- -- -- -- -- -- -- 
20: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
30: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
40: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
50: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
60: -- -- -- -- -- -- -- -- -- -- -- 6b -- -- -- -- 
70: -- -- -- -- -- -- -- --                         
yes: Standardausgabe: Datenübergabe unterbrochen (broken pipe)


In [63]:
print(0x14)

20


In [64]:
print(0x6b)

107


In [68]:
a_star.leds(1, 1, 1)

In [69]:
a_star.read_battery_millivolts()

(7590,)

In [71]:
a_star.read_analog()

(502, 519, 500, 484, 464, 16)

In [73]:
a_star.read_buttons()

(False, False, False)

In [76]:
print(a_star.test_read8())

None


In [89]:
%cd
%cd pololu-rpi-slave-arduino-library/examples
%ls -l */*.*

/home/pirate
/home/pirate/pololu-rpi-slave-arduino-library/examples
-rw-r--r-- 1 pirate pirate 2940 Feb 10 11:12 AStarRPiSlaveDemo/AStarRPiSlaveDemo.ino
-rw-r--r-- 1 pirate pirate 3060 Feb 10 11:12 RomiRPiSlaveDemo/RomiRPiSlaveDemo.ino


In [81]:
# %load AStarRPiSlaveDemo/AStarRPiSlaveDemo.ino
#include <Servo.h>
#include <AStar32U4.h>
#include <PololuRPiSlave.h>

/* This example program shows how to make the A-Star 32U4 Robot
 * Controller into a Raspberry Pi I2C slave.  The RPi and A-Star can
 * exchange data bidirectionally, allowing each device to do what it
 * does best: high-level programming can be handled in a language such
 * as Python on the RPi, while the A-Star takes charge of motor
 * control, analog inputs, and other low-level I/O.
 *
 * The example and libraries are available for download at:
 *
 * https://github.com/pololu/pololu-rpi-slave-arduino-library
 *
 * You will need the corresponding Raspberry Pi code, which is
 * available in that repository under the pi/ subfolder.  The Pi code
 * sets up a simple Python-based web application as a control panel
 * for your Raspberry Pi robot.
 */

// Custom data structure that we will use for interpreting the buffer.
// We recommend keeping this under 64 bytes total.  If you change the
// data format, make sure to update the corresponding code in
// a_star.py on the Raspberry Pi.

struct Data
{
  bool yellow, green, red;
  bool buttonA, buttonB, buttonC;

  int16_t leftMotor, rightMotor;
  uint16_t batteryMillivolts;
  uint16_t analog[6];

  bool playNotes;
  char notes[14];
  
  // Encoders are unused in this example.
  int16_t leftEncoder, rightEncoder;
};

PololuRPiSlave<struct Data,5> slave;
PololuBuzzer buzzer;
AStar32U4Motors motors;
AStar32U4ButtonA buttonA;
AStar32U4ButtonB buttonB;
AStar32U4ButtonC buttonC;

void setup()
{
  // Set up the slave at I2C address 20.
  slave.init(20);

  // Play startup sound.
  buzzer.play("v10>>g16>>>c16");
}

void loop()
{
  // Call updateBuffer() before using the buffer, to get the latest
  // data including recent master writes.
  slave.updateBuffer();

  // Write various values into the data structure.
  slave.buffer.buttonA = buttonA.isPressed();
  slave.buffer.buttonB = buttonB.isPressed();
  slave.buffer.buttonC = buttonC.isPressed();

  // Change this to readBatteryMillivoltsLV() for the LV model.
  slave.buffer.batteryMillivolts = readBatteryMillivoltsSV();

  for(uint8_t i=0; i<6; i++)
  {
    slave.buffer.analog[i] = analogRead(i);
  }

  // READING the buffer is allowed before or after finalizeWrites().
  ledYellow(slave.buffer.yellow);
  ledGreen(slave.buffer.green);
  ledRed(slave.buffer.red);
  motors.setSpeeds(slave.buffer.leftMotor, slave.buffer.rightMotor);

  // Playing music involves both reading and writing, since we only
  // want to do it once.
  static bool startedPlaying = false;
  
  if(slave.buffer.playNotes && !startedPlaying)
  {
    buzzer.play(slave.buffer.notes);
    startedPlaying = true;
  }
  else if (startedPlaying && !buzzer.isPlaying())
  {
    slave.buffer.playNotes = false;
    startedPlaying = false;
  }

  // When you are done WRITING, call finalizeWrites() to make modified
  // data available to I2C master.
  slave.finalizeWrites();
}


In [83]:
# %load RomiRPiSlaveDemo/RomiRPiSlaveDemo.ino
#include <Servo.h>
#include <Romi32U4.h>
#include <PololuRPiSlave.h>

/* This example program shows how to make the Romi 32U4 Control Board 
 * into a Raspberry Pi I2C slave.  The RPi and Romi 32U4 Control Board can
 * exchange data bidirectionally, allowing each device to do what it
 * does best: high-level programming can be handled in a language such
 * as Python on the RPi, while the Romi 32U4 Control Board takes charge 
 * of motor control, analog inputs, and other low-level I/O.
 *
 * The example and libraries are available for download at:
 *
 * https://github.com/pololu/pololu-rpi-slave-arduino-library
 *
 * You will need the corresponding Raspberry Pi code, which is
 * available in that repository under the pi/ subfolder.  The Pi code
 * sets up a simple Python-based web application as a control panel
 * for your Raspberry Pi robot.
 */

// Custom data structure that we will use for interpreting the buffer.
// We recommend keeping this under 64 bytes total.  If you change the
// data format, make sure to update the corresponding code in
// a_star.py on the Raspberry Pi.

struct Data
{
  bool yellow, green, red;
  bool buttonA, buttonB, buttonC;

  int16_t leftMotor, rightMotor;
  uint16_t batteryMillivolts;
  uint16_t analog[6];

  bool playNotes;
  char notes[14];

  int16_t leftEncoder, rightEncoder;
};

PololuRPiSlave<struct Data,5> slave;
PololuBuzzer buzzer;
Romi32U4Motors motors;
Romi32U4ButtonA buttonA;
Romi32U4ButtonB buttonB;
Romi32U4ButtonC buttonC;
Romi32U4Encoders encoders;

void setup()
{
  // Set up the slave at I2C address 20.
  slave.init(20);

  // Play startup sound.
  buzzer.play("v10>>g16>>>c16");
}

void loop()
{
  // Call updateBuffer() before using the buffer, to get the latest
  // data including recent master writes.
  slave.updateBuffer();

  // Write various values into the data structure.
  slave.buffer.buttonA = buttonA.isPressed();
  slave.buffer.buttonB = buttonB.isPressed();
  slave.buffer.buttonC = buttonC.isPressed();

  // Change this to readBatteryMillivoltsLV() for the LV model.
  slave.buffer.batteryMillivolts = readBatteryMillivolts();

  for(uint8_t i=0; i<6; i++)
  {
    slave.buffer.analog[i] = analogRead(i);
  }

  // READING the buffer is allowed before or after finalizeWrites().
  ledYellow(slave.buffer.yellow);
  ledGreen(slave.buffer.green);
  ledRed(slave.buffer.red);
  motors.setSpeeds(slave.buffer.leftMotor, slave.buffer.rightMotor);

  // Playing music involves both reading and writing, since we only
  // want to do it once.
  static bool startedPlaying = false;
  
  if(slave.buffer.playNotes && !startedPlaying)
  {
    buzzer.play(slave.buffer.notes);
    startedPlaying = true;
  }
  else if (startedPlaying && !buzzer.isPlaying())
  {
    slave.buffer.playNotes = false;
    startedPlaying = false;
  }

  slave.buffer.leftEncoder = encoders.getCountsLeft();
  slave.buffer.rightEncoder = encoders.getCountsRight();

  // When you are done WRITING, call finalizeWrites() to make modified
  // data available to I2C master.
  slave.finalizeWrites();
}


In [86]:
%cd
%cd pololu-rpi-slave-arduino-library/src
%ls -l 

/home/pirate
/home/pirate/pololu-rpi-slave-arduino-library/src
insgesamt 16
-rw-r--r-- 1 pirate pirate 4139 Feb 10 11:12 PololuRPiSlave.h
-rw-r--r-- 1 pirate pirate 2604 Feb 10 11:12 PololuTWISlave.cpp
-rw-r--r-- 1 pirate pirate 1446 Feb 10 11:12 PololuTWISlave.h


In [87]:
# %load PololuRPiSlave.h
// Copyright Pololu Corporation.  For more information, see https://www.pololu.com/

#pragma once
#include "PololuTWISlave.h"

/* PololuRPiSlave is an extension of PololuTWISlave that slows down
 * communication where necessary to work around the RPi I2C clock
 * stretching bug described here:
 *
 * http://www.advamation.com/knowhow/raspberrypi/rpi-i2c-bug.html
 *
 * The second template parameter, pi_delay_us, specifies the delay.
 * We recommend a value of 10 for an I2C speed of 100 kHz or a value
 * of 0 for 400 kHz. However, on the Pi 3, CPU scaling will cause I2C
 * to run at half the speed; in this case we recommend values of 20 or
 * 5.
 *
 * Additionally, it implements a system of buffers allowing user code
 * and the I2C system to read and write asynchronously from the same
 * data, without dictating any particular protocol.
 *
 * The data size is determined by the template parameter BufferType.
 * As described below, we allocate four copies of the buffer.  We
 * recommend keeping the buffer under 64 bytes.
 *
 * I2C writes are limited in the code to 16 bytes.
 *
 * You probably don't have to worry about the details below, since the
 * point of this buffering is to make it simple: all you need to do is
 * call updateBuffer() before using the buffer, do your writes and
 * reads, then call finalizeWrites() when you are done.  The I2C
 * master can read and write to the same data at any time, and you
 * should never encounter inconsistent data unless both sides attempt
 * to write to the same region simultaneously.
 *
 * Buffering details:
 *
 * The point is that reads and writes involving I2C and user code are
 * asynchronous and slow, but we want these slow operations to be
 * effectively atomic, so the two sides have to avoid reading and
 * writing from the same buffer at the same time.
 *
 * There is a central buffer (staging_buffer) that is synchronized
 * with three other buffers (buffer, buffer_old, i2c_read_buffer) when
 * appropriate; I2C reads are done directly from i2c_read_buffer, and
 * user code can read and write to "buffer" as desired.
 *
 * There is also a 16-byte buffer i2c_write_buffer, which stores
 * incoming I2C writes until they can be applied.
 */


template <class BufferType, unsigned int pi_delay_us>
  class PololuRPiSlave: public PololuTWISlave
{
private:
  uint8_t index;
  bool index_set = 0;
  uint8_t i2c_write_length = 0;
  uint8_t i2c_write_buffer[16];

  BufferType i2c_read_buffer;
  BufferType staging_buffer;
  BufferType buffer_old;

  void piDelay()
  {
    delayMicroseconds(pi_delay_us);
  }

  void updateI2CBuffer()
  {
    memcpy(&i2c_read_buffer, &staging_buffer, sizeof(BufferType));
  }

  void finalizeI2CWrites()
  {
    if(i2c_write_length == 0) return;

    for(uint8_t i=0; i < i2c_write_length; i++)
    {
      ((uint8_t *)&staging_buffer)[i+index] = i2c_write_buffer[i];
    }
    i2c_write_length = 0;
  }

public:

  BufferType buffer;

  void updateBuffer()
  {
    cli();
    memcpy(&buffer, &staging_buffer, sizeof(BufferType));
    sei();
    memcpy(&buffer_old, &buffer, sizeof(BufferType));
  }

  void finalizeWrites()
  {
    uint8_t i;
    cli();
    for(i=0; i < sizeof(BufferType); i++)
    {
      if(((uint8_t *)&buffer_old)[i] != ((uint8_t *)&buffer)[i])
        ((uint8_t *)&staging_buffer)[i] = ((uint8_t *)&buffer)[i];
    }
    sei();
  }

  virtual void receive(uint8_t b)
  {
    piDelay();
    if(!index_set)
    {
      updateI2CBuffer();
      index = b;
      index_set = true;
    }
    else
    {
      // Wrap writes at the end of the buffer
      if(i2c_write_length > sizeof(i2c_write_buffer))
        i2c_write_length = 0;

      // Write the data to the buffer
      i2c_write_buffer[i2c_write_length] = b;
      i2c_write_length ++;
    }
  }

  virtual uint8_t transmit()
  {
    piDelay();
    return ((uint8_t *)&i2c_read_buffer)[index++];
  }

  virtual void start()
  {
    piDelay();
    index_set = false;
  }

  virtual void stop()
  {
    finalizeI2CWrites();
  }

  /* Initialize the slave on a given address. */
  void init(uint8_t address)
  {
    PololuTWISlave::init(address, *this);
  }
};


In [88]:
# %load PololuTWISlave.h
// Copyright Pololu Corporation.  For more information, see https://www.pololu.com/

#pragma once
#include <stdint.h>

/* PololuTWISlave is a basic AVR I2C slave library that is lightweight
 * and fast.  Unlike the standard Arduino library Wire.h, it does not
 * enforce a particular style of buffering the data - you get to
 * handle the bytes and events one at a time.
 *
 * To use this library, inherit from PololuTWISlave and implement the
 * four virtual functions that specify how to receive and transmit
 * bytes and how to handle the start and stop signals.
 *
 * The library does not support master mode, general calls, error
 * states, and possibly other features of I2C - it only does the
 * minimum required to establish communication with a master that we
 * control.
 */

class PololuTWISlave
{
public:
  /* Methods for a slave to declare.  These methods will be called
   * from the ISR, with clock stretching used to delay further bus
   * activity until they return. */
  virtual void receive(uint8_t b) = 0;
  virtual uint8_t transmit() = 0;
  virtual void start() = 0;
  virtual void stop() = 0;

  /* Initialize slave on a specific address; do not respond to general calls. */
  static void init(uint8_t address, PololuTWISlave &slave);

  /* Low-level static methods not meant to be called by users. */
  static uint8_t handleEvent(uint8_t event);
  static void ack();
  static void nack();
  static void clearBusError();
};
